# interactive_confusion_matrix.ipynb

This notebook should be run within the fifty_one virtual environment.

Activate using:
````
source .venv/bin/activate
````

## References

- https://docs.voxel51.com/user_guide/evaluation.html#evaluating-models
- https://docs.voxel51.com/user_guide/plots.html#confusion-matrices

In [1]:
import fiftyone as fo
from fiftyone import ViewField as F
from ultralytics import YOLO

In [2]:
ROOT_DIR = '/home/aubrey/Desktop/crb-damage-detection-models'
DATASET_DIR = f'{ROOT_DIR}/datasets/CRB005'
MODEL_DIR = f'{ROOT_DIR}/models/CRB005/weights/CRB005.pt'

In [3]:
# List all local datasets by name

# Reports error 26 on first try, indicating a problem opening the Mongo database.
# The following hack runs the code a second time. Seems to work well.
# Note: A dataset can be deleted using fo.delete_dataset("dataset name")
try:
    dataset_list = fo.list_datasets()
except Exception as e:
    print(f'ERROR: {e}')
    print("Retrying...")
    dataset_list = fo.list_datasets()
print(f'Local FiftyOne datasets: {dataset_list}')

{"t":{"$date":"2025-10-29T07:21:21.930Z"},"s":"I",  "c":"CONTROL",  "id":20697,   "ctx":"main","msg":"Renamed existing log file","attr":{"oldLogPath":"/home/aubrey/.fiftyone/var/lib/mongo/log/mongo.log","newLogPath":"/home/aubrey/.fiftyone/var/lib/mongo/log/mongo.log.2025-10-29T07-21-21"}}


Subprocess ['/home/aubrey/Desktop/fifty_one/.venv/lib/python3.12/site-packages/fiftyone/db/bin/mongod', '--dbpath', '/home/aubrey/.fiftyone/var/lib/mongo', '--logpath', '/home/aubrey/.fiftyone/var/lib/mongo/log/mongo.log', '--port', '0', '--nounixsocket'] exited with error 62:


ERROR: fiftyone.core.service.DatabaseService failed to bind to port
Retrying...
Local FiftyOne datasets: ['2025.10.27.15.48.05.506732', 'CRB002s']


In [4]:
fo.delete_non_persistent_datasets()

In [5]:
# Create a new FiftyOne dataset from YOLOv5 formatted data
dataset = fo.Dataset()
for split in ['train', 'val']:
    dataset.add_dir(dataset_dir=DATASET_DIR, dataset_type=fo.types.YOLOv5Dataset, split=split, tags=[split])
print(dataset)

 100% |█████████████████| 104/104 [159.5ms elapsed, 0s remaining, 652.0 samples/s]     
 100% |███████████████████| 47/47 [62.6ms elapsed, 0s remaining, 751.1 samples/s]    
Name:        2025.10.29.15.22.20.832488
Media type:  image
Num samples: 151
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


In [6]:
# Add detections  
print(f'--- Adding detections using YOLOv11 model:\n{MODEL_DIR}')  
model = YOLO(MODEL_DIR)
dataset.apply_model(model, progress=True)
# print summary info about the dataset
print(dataset)
print(f'\ntags: {dataset.distinct("tags")}\n\n')

--- Adding detections using YOLOv11 model:
/home/aubrey/Desktop/crb-damage-detection-models/models/CRB005/weights/CRB005.pt
 100% |█████████████████| 151/151 [5.6s elapsed, 0s remaining, 38.6 samples/s]      
Name:        2025.10.29.15.22.20.832488
Media type:  image
Num samples: 151
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

tags: ['train', 'val']




In [7]:
results = dataset.evaluate_detections(pred_field='predictions', gt_field='ground_truth', eval_key='eval')
session = fo.launch_app(dataset, auto=False)

Evaluating detections...
 100% |█████████████████| 151/151 [1.1s elapsed, 0s remaining, 135.4 samples/s]         
Session launched. Run `session.show()` to open the App in a cell output.



Could not connect session, trying again in 10 seconds



## At this point, the FiftyOne app should be avaialable in a browser at http://localhost:5151

In [8]:
counts = dataset.count_values("ground_truth.detections.label")
classes = sorted(counts, key=counts.get, reverse=True)
results.print_report(classes=classes)

              precision    recall  f1-score   support

     healthy       0.90      0.79      0.84       163
     damaged       0.89      0.83      0.86       143
        vcut       0.89      0.72      0.79       113
        dead       0.87      0.91      0.89       102

   micro avg       0.89      0.81      0.85       521
   macro avg       0.89      0.81      0.85       521
weighted avg       0.89      0.81      0.84       521



In [9]:
import fiftyone as fo
from fiftyone import ViewField as F

# Create a view that has samples with the most false positives first, and
# only includes false positive boxes in the `predictions` field
view = (
    dataset
    .sort_by("eval_fp", reverse=True)
    .filter_labels("predictions", F("eval") == "fp")
)

# Visualize results in the App
session = fo.launch_app(view=view)


Could not connect session, trying again in 10 seconds



RuntimeError: Failed to post event `state_update` to http://localhost:5151/event

In [ ]:
# Plot confusion matrix
plot = results.plot_confusion_matrix(classes=classes)
plot.show()

session.plots.attach(plot)

In [ ]:
results = dataset.evaluate_detections(
    "predictions", 
    gt_field="ground_truth", 
    eval_key="eval"
)

In [ ]:
# Since these results have an `eval_key`, selecting cells in this plot will
# load evaluation patch views
plot = results.plot_confusion_matrix(classes=classes)
plot.show(height=600)

session.plots.attach(plot)

In [10]:
dataset.persistent = True
dataset.save()

In [11]:
dataset

Name:        2025.10.29.15.22.20.832488
Media type:  image
Num samples: 151
Persistent:  True
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:          fiftyone.core.fields.IntField
    eval_fp:          fiftyone.core.fields.IntField
    eval_fn:          fiftyone.core.fields.IntField